## Imports

In [3]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup 
import re
import pandas as pd 
import os 
import time
import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##### Introducing Selenium 

In [2]:
#The 7th: http://www.santaanita.com/live-racing/4718/
#The 6th: http://www.santaanita.com/live-racing/4618/


## Horses of Interest

In [3]:
mainpage = "https://www.equibase.com/static/chart/summary/SA040718USA-EQB.html"
horselink = BeautifulSoup(requests.get(mainpage).text, "lxml")


In [22]:
###New list of horses!! Race 7 and 10.
horsenames = ['Accountability','Avalanche','Fortune of War', 'Mesut','Elwood J','Tartini','Dreams of Valor','Cascade Rock', 'Malibu Music',
              'Beau Recall','Thundering Sky','Madame Stripes','Enola Grey','Sassy Little Lila','Sophie P','Instant Reflex','Midnight Crossing',
              'Mongolian Shopper']

In [23]:
horselinkdic = {}

for horse in horsenames:
    for a in horselink.find_all("a", text=re.compile(horse)):
        for i in a:
            a = a['href']
            horselinkdic[horse]=a
            

NameError: name 'horselink' is not defined

In [6]:
print((horselinkdic))

{'Accountability': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9700631&registry=T&rbt=TB', 'Avalanche': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9694429&registry=T&rbt=TB', 'Fortune of War': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9732830&registry=T&rbt=TB', 'Mesut': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9708093&registry=T&rbt=TB', 'Elwood J': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9691578&registry=T&rbt=TB', 'Tartini': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9694778&registry=T&rbt=TB', 'Dreams of Valor': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9649893&registry=T&rbt=TB', 'Cascade Rock': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9789258&registry=T&rbt=TB', 'Malibu Music': 'http://www.equibase.com/profiles/Results.cfm?type=Horse&refno=9762146&registry=T&rbt=TB', 'Beau Recall': 'http://www.equibase.com/profiles

## Scraping horsey stats

##### Scraping the stats table

In [15]:
#launch url
chromedriver = "/Users/vicky/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [16]:
horseywinnings=[]
horseyworkouts=[]

for key, value in horselinkdic.items():   
    
    print(key) #So when it inevitably glitches in the middle I know which horse is giving me trouble. 
    jesustakethewheel = webdriver.Chrome(chromedriver)
    jesustakethewheel.get(value)
    
    soupresults=BeautifulSoup(requests.get(value).text, "lxml")
    resultstable=soupresults.find_all(class_='table-hover')[1]

    rows=[row for row in resultstable.find_all('tr')]
    rows=rows[1:5]


    horseywins={}
    for row in rows:
        items=row.find_all('td')
        for entries in items:
            splitsies = entries.text.split(':')
            horseywins[splitsies[0]] = (splitsies[1])
            horseywins['Horse Name'] = (key)    
    
    
    horseywinnings.append(horseywins)
    
    time.sleep(1)
    
    python_button_Workouts = jesustakethewheel.find_element_by_id('Hworkouts') 
    python_button_Workouts.click() 
    time.sleep(.5)
    
    python_button_seemore = jesustakethewheel.find_element_by_link_text('SEE MORE WORKOUTS')
    python_button_seemore.click()
    time.sleep(.5)
    
    soupworkout=BeautifulSoup(jesustakethewheel.page_source, "lxml")
    
    workouttable=soupworkout.find(class_='resultTable') 

    #horseyworkouts=[]
    for row in workouttable.find_all('tr')[1:]:
        items=row.find_all('td')
        Track = items[0].text
        Date = items[1].text
        Course = items[2].text
        Distance = items[3].text
        if len(items[4].text) <6:
            tp = datetime.datetime.strptime(items[4].text,'%S.%f')
        else:
            tp = datetime.datetime.strptime(items[4].text,'%M:%S.%f')
        Time = tp.second*10+tp.minute*600+tp.microsecond//100000
        Note = items[5].text
        Rank = items[6].text
        rowdict= {'Horse Name': key,'Course' :Course,'Track' :Track,'Date':Date ,'Course' :Course ,'Distance' :Distance,'Time_tenths_second':Time,'Note':Note,'Rank':Rank}
        horseyworkouts.append(rowdict)
            
    time.sleep(1)

Accountability
Avalanche
Fortune of War
Mesut
Elwood J
Tartini
Dreams of Valor
Cascade Rock
Malibu Music
Beau Recall
Thundering Sky
Madame Stripes
Sassy Little Lila
Sophie P
Instant Reflex
Midnight Crossing
Mongolian Shopper


In [17]:
print(horseyworkouts)

[{'Horse Name': 'Accountability', 'Course': 'Dirt', 'Track': ' Santa Anita', 'Date': '03/31/2018', 'Distance': '5F', 'Time_tenths_second': 610, 'Note': 'Handily', 'Rank': '\n                                18/70 \n                            '}, {'Horse Name': 'Accountability', 'Course': 'Dirt', 'Track': ' Santa Anita', 'Date': '03/25/2018', 'Distance': '5F', 'Time_tenths_second': 620, 'Note': 'Handily', 'Rank': '\n                                62/110 \n                            '}, {'Horse Name': 'Accountability', 'Course': 'Dirt', 'Track': ' Santa Anita', 'Date': '03/20/2018', 'Distance': '3F', 'Time_tenths_second': 378, 'Note': 'Handily', 'Rank': '\n                                22/40 \n                            '}, {'Horse Name': 'Accountability', 'Course': 'Dirt', 'Track': ' Santa Anita', 'Date': '03/05/2018', 'Distance': '5F', 'Time_tenths_second': 624, 'Note': 'Handily', 'Rank': '\n                                97/114 \n                            '}, {'Horse Name': 'A

In [4]:
df_stats2 = pd.DataFrame(horseywinnings)
dfworkouts2 = pd.DataFrame(horseyworkouts)

NameError: name 'horseywinnings' is not defined

In [19]:
df_stats2.to_csv('stats_csv_2')
dfworkouts2.to_csv('workouts_csv_2')

## Cleaning up dfworkouts

In [8]:
dfworkouts2 = pd.read_csv('workouts_csv_2')

In [9]:
dfworkouts2['Course'].unique()

array(['Dirt', 'Dirt training', 'Turf', 'Turf training', 'Inner turf',
       'All Weather Track', 'All Weather Training'], dtype=object)

In [11]:
dfworkouts_clean2 = dfworkouts2.drop('Note', axis=1 ).drop('Rank',axis=1).drop('Track',axis=1)
#new_df = df[df.C != 'XYZ']
#dfworkouts_clean = dfworkouts_clean.drop(dfworkouts_clean.loc[dfworkouts_clean['Course'] == 'Turf'])

In [12]:
dfworkouts_clean2.head(2)

,Unnamed: 0,Course,Date,Distance,Horse Name,Time_tenths_second
0,0,Dirt,03/31/2018,5F,Accountability,610
1,1,Dirt,03/25/2018,5F,Accountability,620


### GROUP BY TO AVERAGE

In [ ]:
dfworkouts_clean.groupby

## Cleaning up df_stats

In [13]:
df_stats

NameError: name 'df_stats' is not defined

## Race times and Gender df

In [4]:
resultsofinterest = ['{Horse Name:  , Final_Time}',
                    'http://tnetwork.trakus.com/tnet/t_SantaAnita.aspx?EventID=133974&Date=4/14/2018&Type=TBRED&Venue=4&DisplayType=1',
                    'http://tnetwork.trakus.com/tnet/t_SantaAnita.aspx?EventID=133976&Date=4/14/2018&Type=TBRED&Venue=4&DisplayType=1']

In [11]:
timesdic = [{'Horse Name' : 'Accountability', 'Final_Time':'1:34.39','Gender':'0'},
           {'Horse Name' :'Avalanche','Final_Time': '1:34.47','Gender': '0'},
           {'Horse Name' :'Fortune of War','Final_Time': '1:34.54','Gender':'0'},
           {'Horse Name' :'Mesut','Final_Time': '1:34.77','Gender':'0'},
           {'Horse Name' :'Elwood J','Final_Time':'1:34.91','Gender':'0'},
           {'Horse Name' :'Tartini','Final_Time': '1:35.06','Gender':'0'},
           {'Horse Name' :'Dreams of Valor','Final_Time': '1:35.16','Gender':'0'},
           {'Horse Name' :'Cascade Rock','Final_Time': '1:35.45','Gender':'0'},
           {'Horse Name' :'Malibu Music','Final_Time': '1:36.85','Gender':'0'},
           {'Horse Name' :'Beau Recall','Final_Time': '1:34.52','Gender':'1'},
           {'Horse Name' :'Thundering Sky','Final_Time':'1:34.64','Gender':'1'},
           {'Horse Name' :'Madame Stripes','Final_Time': '1:34.68','Gender':'1'},
           {'Horse Name' :'Sassy Little Lila','Final_Time':'1:34.99','Gender':'1'},
           {'Horse Name' :'Sophie P','Final_Time': '1:35.03','Gender':'1'},
           {'Horse Name' :'Instant Reflex','Final_Time':'1:35.16','Gender':'1'},
           {'Horse Name' :'Midnight Crossing','Final_Time': '1:35.17','Gender':'1'},
           {'Horse Name' :'Mongolian Shopper','Final_Time': '1:36.33','Gender':'1'}]


In [12]:

for row in timesdic:
    for k,v in row.items():
        if k == 'Final_Time':
            timing = datetime.datetime.strptime(v,'%M:%S.%f')
            value = timing.second*100+timing.minute*6000+timing.microsecond//10000
        
    
    row['Final_Time_Hund'] = value 

print(timesdic)
    
    

[{'Horse Name': 'Accountability', 'Final_Time': '1:34.39', 'Gender': '0', 'Final_Time_Hund': 9439}, {'Horse Name': 'Avalanche', 'Final_Time': '1:34.47', 'Gender': '0', 'Final_Time_Hund': 9447}, {'Horse Name': 'Fortune of War', 'Final_Time': '1:34.54', 'Gender': '0', 'Final_Time_Hund': 9454}, {'Horse Name': 'Mesut', 'Final_Time': '1:34.77', 'Gender': '0', 'Final_Time_Hund': 9477}, {'Horse Name': 'Elwood J', 'Final_Time': '1:34.91', 'Gender': '0', 'Final_Time_Hund': 9491}, {'Horse Name': 'Tartini', 'Final_Time': '1:35.06', 'Gender': '0', 'Final_Time_Hund': 9506}, {'Horse Name': 'Dreams of Valor', 'Final_Time': '1:35.16', 'Gender': '0', 'Final_Time_Hund': 9516}, {'Horse Name': 'Cascade Rock', 'Final_Time': '1:35.45', 'Gender': '0', 'Final_Time_Hund': 9545}, {'Horse Name': 'Malibu Music', 'Final_Time': '1:36.85', 'Gender': '0', 'Final_Time_Hund': 9685}, {'Horse Name': 'Beau Recall', 'Final_Time': '1:34.52', 'Gender': '1', 'Final_Time_Hund': 9452}, {'Horse Name': 'Thundering Sky', 'Final_Ti

In [13]:
dffinaltime2 = pd.DataFrame.from_dict(timesdic)

#dfpract = dfpract.transpose()
dffinaltime2

,Final_Time,Final_Time_Hund,Gender,Horse Name
0,1:34.39,9439,0,Accountability
1,1:34.47,9447,0,Avalanche
2,1:34.54,9454,0,Fortune of War
3,1:34.77,9477,0,Mesut
4,1:34.91,9491,0,Elwood J
5,1:35.06,9506,0,Tartini
6,1:35.16,9516,0,Dreams of Valor
7,1:35.45,9545,0,Cascade Rock
8,1:36.85,9685,0,Malibu Music
9,1:34.52,9452,1,Beau Recall


# Rereading df_stats2 to act on it

In [18]:
df_stats2 = pd.read_csv('stats_csv_2').drop('Unnamed: 0', axis=1)

In [19]:
df_stats2

,Firsts,Horse Name,Seconds,Starts,Thirds
0,3,Accountability,0,14,3
1,1,Avalanche,2,8,1
2,1,Fortune of War,3,16,4
3,1,Mesut,3,7,0
4,2,Elwood J,1,16,5
5,1,Tartini,1,9,1
6,1,Dreams of Valor,0,10,0
7,1,Cascade Rock,3,10,1
8,1,Malibu Music,2,13,1
9,3,Beau Recall,4,12,0


In [37]:
Race_seven = ['Accountability'1,'Avalanche'2,'Fortune of War'3, 'Mesut','Elwood J','Tartini','Dreams of Valor','Cascade Rock', 'Malibu Music']
#Allowance optional claiming
Race_ten = ['Beau Recall'1,'Thundering Sky'2,'Madame Stripes'3,'Sassy Little Lila','Sophie P','Instant Reflex','Midnight Crossing',
              'Mongolian Shopper']
#Stakes

## Merging final time and df stats to clean

In [20]:
finalmerged2 = pd.merge(dffinaltime2,df_stats2,on='Horse Name', how = 'left')

In [21]:
finalmerged2['Starts'] = finalmerged2['Starts'] - 1
finalmerged2


,Final_Time,Final_Time_Hund,Gender,Horse Name,Firsts,Seconds,Starts,Thirds
0,1:34.39,9439,0,Accountability,3,0,13,3
1,1:34.47,9447,0,Avalanche,1,2,7,1
2,1:34.54,9454,0,Fortune of War,1,3,15,4
3,1:34.77,9477,0,Mesut,1,3,6,0
4,1:34.91,9491,0,Elwood J,2,1,15,5
5,1:35.06,9506,0,Tartini,1,1,8,1
6,1:35.16,9516,0,Dreams of Valor,1,0,9,0
7,1:35.45,9545,0,Cascade Rock,1,3,9,1
8,1:36.85,9685,0,Malibu Music,1,2,12,1
9,1:34.52,9452,1,Beau Recall,3,4,11,0


In [24]:
#Setting index so we can subtract 1st, 2nd, 3rd place counts for that day's race. 
finalmerged2 = ( finalmerged2.set_index(['Horse Name']))
finalmerged2

,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds
Horse Name,,,,,,,
Accountability,1:34.39,9439,0,3,0,13,3
Avalanche,1:34.47,9447,0,1,2,7,1
Fortune of War,1:34.54,9454,0,1,3,15,4
Mesut,1:34.77,9477,0,1,3,6,0
Elwood J,1:34.91,9491,0,2,1,15,5
Tartini,1:35.06,9506,0,1,1,8,1
Dreams of Valor,1:35.16,9516,0,1,0,9,0
Cascade Rock,1:35.45,9545,0,1,3,9,1
Malibu Music,1:36.85,9685,0,1,2,12,1


In [30]:
finalmerged2.loc[['Accountability','Beau Recall'], ['Firsts']] = finalmerged2.loc[['Accountability','Beau Recall'], ['Firsts']]-1

In [31]:
finalmerged2.loc[['Avalanche','Thundering Sky'], ['Seconds']] = finalmerged2.loc[['Avalanche','Thundering Sky'], ['Seconds']]-1

In [32]:
finalmerged2.loc[['Fortune of War','Madame Stripes'], ['Thirds']] = finalmerged2.loc[['Fortune of War','Madame Stripes'], ['Thirds']]-1

In [33]:
finalmerged2

,Final_Time,Final_Time_Hund,Gender,Firsts,Seconds,Starts,Thirds
Horse Name,,,,,,,
Accountability,1:34.39,9439,0,2,0,13,3
Avalanche,1:34.47,9447,0,1,1,7,1
Fortune of War,1:34.54,9454,0,1,3,15,3
Mesut,1:34.77,9477,0,1,3,6,0
Elwood J,1:34.91,9491,0,2,1,15,5
Tartini,1:35.06,9506,0,1,1,8,1
Dreams of Valor,1:35.16,9516,0,1,0,9,0
Cascade Rock,1:35.45,9545,0,1,3,9,1
Malibu Music,1:36.85,9685,0,1,2,12,1


In [34]:
finalmerged2.to_csv('final_merged2')